In [17]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.mixture import GaussianMixture
from scipy.stats import multivariate_normal
from scipy.stats import ortho_group

In [53]:
# utility function to generate random non singular covariance matrices with 
def generate_covariance_matrices(n_components, d):
    ans = []
    for i in range(n_components):
        Q = ortho_group.rvs(d)
        decay = 0.4*(1 + np.random.random())
        eps = 1e-8
        U = np.ones(d,) * (np.random.rand() * 20) + 600
        for j in range(1,d):
            U[j] = max((U[j-1] * decay) + (np.random.rand() * 0.01), eps)
        ans.append(Q @ np.diag(U) @ Q.transpose())
    return ans

In [35]:
# gmm recovery from a synthetic gmm
# make a representative gmm
def make_gmm(n_components, n_features, random_state=0):
    gmm = GaussianMixture(n_components=n_components, random_state=random_state)
    gmm.means_ = np.random.rand(n_components, n_features) * 4
    covariance_array = generate_covariance_matrices(n_components, n_features)
    # for i in range(n_components):
    #     rand_matrix = np.random.rand(n_features, n_features)
    #     covariance_array.append(np.dot(rand_matrix, rand_matrix.T))

    gmm.covariances_ = np.array(covariance_array)

    precision_array = [np.linalg.pinv(cov) for cov in covariance_array]
    gmm.precisions_ = np.array(precision_array)
    gmm.precisions_cholesky_ = np.array([np.linalg.cholesky(prec) for prec in precision_array])
    
    gmm.weights_ = np.random.rand(n_components)
    gmm.weights_ /= np.sum(gmm.weights_)
    return gmm

In [36]:
n_components = 5
n_features = 20

my_test_gmm = make_gmm(n_components, n_features)
my_test_gmm.sample(1)

(array([[ -5.26836486,  -7.88965868,   6.11643519, -11.45308188,
           3.66742442,   7.74727604,  12.75144606,   2.19364309,
          -8.06624039,   7.58596306,  -1.22982727, -25.68076378,
         -10.86052499,  -3.29374238,   0.95878925,  18.40788379,
         -35.95657899,  -8.66452726,   3.78172571,  -5.65498484]]),
 array([1]))

In [37]:
# take compressive meaurements
# make 10 random measurement matrices
n_measurements = ((n_features * 7) // 10) # 50% of the features

measurement_matrices = []
for i in range(10):
    # measurement_matrices.append(np.random.randn(n_measurements, n_features))
    measurement_matrices.append(np.random.binomial(1, 0.5, size=(n_measurements, n_features)))

# generate samples from the gmm
n_samples = 2000
samples = my_test_gmm.sample(n_samples)

# take measurements
compressed_measurements = []
sample_measurement_matrix = []
for i in range(n_samples):
    matrix_index = np.random.randint(0, len(measurement_matrices))
    sample_measurement_matrix.append(matrix_index)
    compressed_measurements.append(np.dot(measurement_matrices[matrix_index], samples[0][i]))


noise_std_dev = 0.0005
noise_covariance_matrix = np.eye(n_measurements) * (noise_std_dev * noise_std_dev)
compressed_measurements = np.array(compressed_measurements)
comp_copy = compressed_measurements.copy()
compressed_measurements += np.random.normal(0, noise_std_dev, compressed_measurements.shape)
print(compressed_measurements.shape)
print(samples[0].shape)
print(measurement_matrices[0].shape)
print(len(sample_measurement_matrix))

diff = 0
for i in range(compressed_measurements.shape[0]):
    diff += np.linalg.norm(compressed_measurements[i] - comp_copy[i])

diff /= compressed_measurements.shape[0]
print(diff)

# my_test_gmm = GaussianMixture(n_components=n_components,  n_init=5, verbose=1, max_iter=200, init_params='random')
# my_test_gmm.fit(samples[0])

(2000, 14)
(2000, 20)
(14, 20)
2000
0.0018302139126426498


In [38]:
# for each of the 10 measurement matrices we have a separate gmm in the y-domain so let us make all of them p(y|z)

def form_y_gmms(x_gmm, measurement_matrices, noise_covariance_matrix):
    gmm_list = []
    noise_inverse = np.linalg.inv(noise_covariance_matrix)
    for i in range(len(measurement_matrices)):
        gmm_sample = GaussianMixture(n_components=n_components, random_state=0)
        gmm_sample.means_ = x_gmm.means_ @ measurement_matrices[i].T
        covariance_array = []

        for j in range(n_components):
            # C_ij = np.linalg.pinv(measurement_matrices[i].T @ noise_inverse @ measurement_matrices[i] + x_gmm.precisions_[j])
            # temp_matrix = noise_inverse @ measurement_matrices[i] @ C_ij @ measurement_matrices[i].T @ noise_inverse
            # covariance_array.append(np.linalg.pinv(noise_inverse - temp_matrix))
            covariance_array.append(noise_covariance_matrix  + measurement_matrices[i] @ x_gmm.covariances_[j] @ measurement_matrices[i].T)

        gmm_sample.covariances_ = np.array(covariance_array)
        gmm_sample.weights_ = x_gmm.weights_

        precision_array = [np.linalg.pinv(covariance_array[i]) for i in range(n_components)]
        gmm_sample.precisions_ = np.array(precision_array)
        gmm_sample.precisions_cholesky_ = np.array([np.linalg.cholesky(precision_array[i]) for i in range(n_components)])
        
        gmm_list.append(gmm_sample)
    return gmm_list


In [39]:
def get_log_likelihood(measurement_matrices, compressed_measurements, sample_measurement_matrix, gmm_list):
    log_likelihood = 0
    for i in range(compressed_measurements.shape[0]):
        # print(gmm_list[sample_measurement_matrix[i]].score(compressed_measurements[i].reshape(1, -1)))
        log_likelihood += gmm_list[sample_measurement_matrix[i]].score(compressed_measurements[i].reshape(1, -1))

    return log_likelihood/len(compressed_measurements)

In [40]:
def get_new_weights(compressed_measurements, y_gmms, sample_measurement_matrix):
    # calculate p_ik
    new_weights = []
    for k in range(n_components):
        ssum = 0
        for i in range(n_samples):
            p_ik = y_gmms[sample_measurement_matrix[i]].weights_[k] * multivariate_normal.pdf(compressed_measurements[i], mean=y_gmms[sample_measurement_matrix[i]].means_[k], cov=y_gmms[sample_measurement_matrix[i]].covariances_[k]) / np.exp(y_gmms[sample_measurement_matrix[i]].score(compressed_measurements[i].reshape(1, -1)))
            ssum += p_ik
        new_weights.append(ssum)
    new_weights = np.array(new_weights)
    new_weights /= np.sum(new_weights)
    return new_weights

In [41]:
def get_x_reconstruction(measurement_matrix, noise_covariance_matrix, x_covariance_matrix, x_mean, compressed_measurement):
    """
    gets the x reconstruction for a compressed sample for the z^th component in x (specified by x_covariance_matrix and x_mean)
    """
    # noise_inverse = np.linalg.pinv(noise_covariance_matrix)
    # C_z = np.linalg.pinv(measurement_matrix.T @ noise_inverse @ measurement_matrix + x_precision_matrix)
    C_z = np.linalg.inv(noise_covariance_matrix + (measurement_matrix @ x_covariance_matrix @ measurement_matrix.T))
    x_reconstruction = x_mean + (x_covariance_matrix @ measurement_matrix.T @ C_z @ (compressed_measurement - measurement_matrix @ x_mean))
    return x_reconstruction    

In [42]:
def get_new_means(compressed_measurements, y_gmms, sample_measurement_matrix, estimated_gmm, measurement_matrices, noise_covariance_matrix):
    new_means = []
    for k in range(n_components):
        ssum = 0
        psum = 0
        for i in range(n_samples):
            p_ik = y_gmms[sample_measurement_matrix[i]].weights_[k] * multivariate_normal.pdf(compressed_measurements[i], mean=y_gmms[sample_measurement_matrix[i]].means_[k], cov=y_gmms[sample_measurement_matrix[i]].covariances_[k]) / np.exp(y_gmms[sample_measurement_matrix[i]].score(compressed_measurements[i].reshape(1, -1)))
            psum += p_ik
            n_ik = get_x_reconstruction(measurement_matrices[sample_measurement_matrix[i]], noise_covariance_matrix, estimated_gmm.covariances_[k], estimated_gmm.means_[k], compressed_measurements[i])
            ssum += p_ik * n_ik
        new_means.append(ssum/psum)
    new_means = np.array(new_means)
    return new_means

In [43]:
def get_new_covariances(compressed_measurements, y_gmms, sample_measurement_matrix, estimated_gmm, measurement_matrices, noise_covariance_matrix, new_means):
    new_covariances = []
    for k in range(n_components):
        csum = 0
        psum = 0
        for i in range(n_samples):
            measurement_matrix = measurement_matrices[sample_measurement_matrix[i]]
            p_ik = y_gmms[sample_measurement_matrix[i]].weights_[k] * multivariate_normal.pdf(compressed_measurements[i], mean=y_gmms[sample_measurement_matrix[i]].means_[k], cov=y_gmms[sample_measurement_matrix[i]].covariances_[k]) / np.exp(y_gmms[sample_measurement_matrix[i]].score(compressed_measurements[i].reshape(1, -1)))
            psum += p_ik
            n_ik = get_x_reconstruction(measurement_matrices[sample_measurement_matrix[i]], noise_covariance_matrix, estimated_gmm.covariances_[k], estimated_gmm.means_[k], compressed_measurements[i])
            C_ik = estimated_gmm.covariances_[k] - (estimated_gmm.covariances_[k] @ measurement_matrix.T @ np.linalg.inv(noise_covariance_matrix + (measurement_matrix @ estimated_gmm.covariances_[k] @ measurement_matrix.T)) @ measurement_matrix @ estimated_gmm.covariances_[k])
            csum += p_ik * (C_ik + (n_ik - new_means[k]).reshape(-1, 1) @ (n_ik - new_means[k]).reshape(1, -1))
        new_covariances.append(csum/psum)
    new_covariances = np.array(new_covariances)
    return new_covariances

First check if the reconstruction is good using the suggested method (compare Max-Max estimate and MSE optimal estimate)

In [44]:
def weighted_l2(v, mat):
    if np.linalg.matrix_rank(mat) < mat.shape[0]:
        # handles the case where the matrix is not invertible
        m = v.T @ np.linalg.pinv(mat) @ v
    else:
        m = v.T @ np.linalg.inv(mat) @ v
    return m

In [45]:
def decode(model, A, y, noise_covariance_matrix):
    x_hat = np.empty(model.means_.shape)
    cost = []
    var_noise = noise_covariance_matrix

    for j in range(model.means_.shape[0]):
        var_j, mu_j = model.covariances_[j], model.means_[j]
        x_hat_j = var_j @ A.T @ np.linalg.inv(A @ var_j @ A.T + var_noise) @ (y - A @ mu_j) + mu_j
        # print(np.linalg.det(var_j))
        # print(var_j)
        cost_j = weighted_l2(y - A @ x_hat_j, var_noise) + weighted_l2(x_hat_j - mu_j, var_j) + np.log(np.linalg.det(var_j))
        # print(np.linalg.det(var_j))
        x_hat[j] = x_hat_j
        cost.append(cost_j)

    j = np.argmin(cost)
    # print(j)
    return x_hat[j]

In [46]:
# min MSE scheme
reconstructed_samples = []
y_gmms = form_y_gmms(my_test_gmm, measurement_matrices, noise_covariance_matrix)

for i in range(n_samples):
    y_gmm = y_gmms[sample_measurement_matrix[i]]
    reconstructed_sample = np.zeros(my_test_gmm.means_[0].shape)
    psum = 0
    for j in range(n_components):
        p_ik = y_gmm.weights_[j] * multivariate_normal.pdf(compressed_measurements[i], mean=y_gmm.means_[j], cov=y_gmm.covariances_[j]) / np.exp(y_gmm.score(compressed_measurements[i].reshape(1, -1)))
        psum += p_ik
        n_ik = get_x_reconstruction(measurement_matrices[sample_measurement_matrix[i]], noise_covariance_matrix, my_test_gmm.covariances_[j], my_test_gmm.means_[j], compressed_measurements[i])
        reconstructed_sample += p_ik * n_ik
    assert abs(psum - 1) < 1e-3
    reconstructed_samples.append(reconstructed_sample)

reconstructed_samples = np.array(reconstructed_samples)
test_rmse = np.sqrt(np.mean(np.sum((reconstructed_samples - samples[0]) ** 2, axis=1)))
print(f"Min MSE RMSE: {test_rmse}")

# max-max scheme
mm_recon = []
for i in range(n_samples):
    mm_recon.append(decode(my_test_gmm, measurement_matrices[sample_measurement_matrix[i]], compressed_measurements[i], noise_covariance_matrix))
mm_recon = np.array(mm_recon)
mm_rmse = np.sqrt(np.mean(np.sum((mm_recon - samples[0]) ** 2, axis=1)))
print(f"MM RMSE: {mm_rmse}")
    

Min MSE RMSE: 2.20722856689254
MM RMSE: 2.2057181361134495


In [47]:
print(mm_recon[0])
print(reconstructed_samples[0])
print(samples[0][0])

[ 8.21457588  2.92504666 -2.4157243  -0.53222906  8.68822933  2.14010323
 -3.92733048  7.54069754  0.26874633 -3.05725303  0.08985494 -1.7208007
  7.53479582  2.38227858  3.81573647  2.32245132  0.46094808  4.74985201
  7.6257281   2.7774674 ]
[ 8.21457588  2.92504666 -2.4157243  -0.53222906  8.68822933  2.14010323
 -3.92733048  7.54069754  0.26874633 -3.05725303  0.08985494 -1.7208007
  7.53479581  2.38227858  3.81573647  2.32245131  0.46094808  4.74985201
  7.62572809  2.7774674 ]
[ 8.11626355  3.0888958  -2.87547694  0.02393244  8.85621109  1.97785846
 -4.18169281  7.74912808 -0.19325706 -3.52072684 -0.02607054 -1.10046749
  7.01940333  3.1064015   4.82483983  2.46493158  0.05849487  4.70260456
  7.84598354  1.92990974]


In [14]:
# initialise estimate of the original gmm
# assumption is that we know the number of components required to model the original gmm
estimated_gmm = make_gmm(n_components, n_features, random_state=7)

# perform updates on the estimated gmm
# log likelihoods stores the log likelihoods of the estimated gmm computed on the samples drawn from the original gmm, our objective is to maximise this
tol = 0.05
log_likelihoods = []
y_gmms = form_y_gmms(estimated_gmm, measurement_matrices, noise_covariance_matrix)
log_likelihoods.append(get_log_likelihood(measurement_matrices, compressed_measurements, sample_measurement_matrix, y_gmms))

print("Initial log likelihood: ", log_likelihoods[-1])
ideal_log_likelihood = get_log_likelihood(measurement_matrices, compressed_measurements, sample_measurement_matrix, form_y_gmms(my_test_gmm, measurement_matrices, noise_covariance_matrix))
print("Ideal log likelihood: ", ideal_log_likelihood)

noise_inverse =  np.linalg.inv(noise_covariance_matrix)
iter_counter = 0

while True:
    # perform updates
    # for each of the 10 measurement matrices we have a separate gmm in the y-domain so let us make all of them
    # update weights
    # new_weights = []
    # for i in range(n_components):
    #     ssum = 0
    #     for j in range(len(compressed_measurements)):
    #         den = 0
    #         for k in range(n_components):
    #             den += estimated_gmm.weights_[k] * multivariate_normal.pdf(compressed_measurements[j], mean=y_gmms[sample_measurement_matrix[j]].means_[k], cov=y_gmms[sample_measurement_matrix[j]].covariances_[k])
    #         ssum += estimated_gmm.weights_[i] * multivariate_normal.pdf(compressed_measurements[j], mean=y_gmms[sample_measurement_matrix[j]].means_[i], cov=y_gmms[sample_measurement_matrix[j]].covariances_[i]) / den
    #     new_weights.append(ssum) # entry appended was \sum_{i = 1}^{N} p_ik for k as the iter counter (here 'i')
    # new_weights = np.array(new_weights)
    # # new_weights /= np.sum(new_weights) # do this after mean and covariance update

    # # update means
    # new_means = []
    # for i in range(n_components):
    #     ssum = 0
    #     psum = 0
    #     for j in range(len(compressed_measurements)):
    #         den = 0
    #         for k in range(n_components):
    #             den += estimated_gmm.weights_[k] * multivariate_normal.pdf(compressed_measurements[j], mean=y_gmms[sample_measurement_matrix[j]].means_[k], cov=y_gmms[sample_measurement_matrix[j]].covariances_[k])
    #         p_ik = estimated_gmm.weights_[i] * multivariate_normal.pdf(compressed_measurements[j], mean=y_gmms[sample_measurement_matrix[j]].means_[i], cov=y_gmms[sample_measurement_matrix[j]].covariances_[i]) / den
    #         C_ik = np.linalg.pinv(measurement_matrices[sample_measurement_matrix[j]].T @ noise_inverse @ measurement_matrices[sample_measurement_matrix[j]] + estimated_gmm.precisions_[i])
    #         n_ik = estimated_gmm.means_[i] + (C_ik @ measurement_matrices[sample_measurement_matrix[j]].T @ noise_inverse @ (compressed_measurements[j] - measurement_matrices[sample_measurement_matrix[j]] @ estimated_gmm.means_[i]))
    #         ssum += p_ik * n_ik
    #         psum += p_ik
    #     new_means.append(ssum / psum)

    # # update covariances
    # new_covariances = []
    # for i in range(n_components):
    #     ssum = 0
    #     psum = 0
    #     for j in range(len(compressed_measurements)):
    #         den = 0
    #         for k in range(n_components):
    #             den += y_gmms[sample_measurement_matrix[j]].weights_[k] * multivariate_normal.pdf(compressed_measurements[j], mean=y_gmms[sample_measurement_matrix[j]].means_[k], cov=y_gmms[sample_measurement_matrix[j]].covariances_[k])
    #         p_ik = y_gmms[sample_measurement_matrix[j]].weights_[i] * multivariate_normal.pdf(compressed_measurements[j], mean=y_gmms[sample_measurement_matrix[j]].means_[i], cov=y_gmms[sample_measurement_matrix[j]].covariances_[i]) / den
    #         C_ik = np.linalg.pinv((measurement_matrices[sample_measurement_matrix[j]].T @ noise_inverse @ measurement_matrices[sample_measurement_matrix[j]]) + estimated_gmm.precisions_[i])
    #         n_ik = estimated_gmm.means_[i] + C_ik @ (measurement_matrices[sample_measurement_matrix[j]].T @ noise_inverse @ (compressed_measurements[j] - measurement_matrices[sample_measurement_matrix[j]] @ estimated_gmm.means_[i]))
    #         ssum += p_ik * ((n_ik - new_means[i]) @ (n_ik - new_means[i]).T + C_ik)
    #         psum += p_ik
    #     new_covariances.append(ssum / psum)


    estimated_gmm.weights_ = get_new_weights(compressed_measurements, y_gmms, sample_measurement_matrix)
    estimated_gmm.means_ = get_new_means(compressed_measurements, y_gmms, sample_measurement_matrix, estimated_gmm, measurement_matrices, noise_covariance_matrix)
    estimated_gmm.covariances_ = get_new_covariances(compressed_measurements, y_gmms, sample_measurement_matrix, estimated_gmm, measurement_matrices, noise_covariance_matrix, estimated_gmm.means_)
    precision_array = [np.linalg.pinv(cov) for cov in estimated_gmm.covariances_]
    estimated_gmm.precisions_ = np.array(precision_array)
    estimated_gmm.precisions_cholesky_ = np.array([np.linalg.cholesky(prec) for prec in estimated_gmm.precisions_])

    y_gmms = form_y_gmms(estimated_gmm, measurement_matrices, noise_covariance_matrix)
    log_likelihoods.append(get_log_likelihood(measurement_matrices, compressed_measurements, sample_measurement_matrix, y_gmms))
    print(f"Iteration {iter_counter + 1} log-likelihood: {log_likelihoods[-1]}")
    iter_counter += 1
    if abs(log_likelihoods[-1] - log_likelihoods[-2]) < tol or log_likelihoods[-1] > ideal_log_likelihood:
        if abs(log_likelihoods[-1] - log_likelihoods[-2]) < tol:
            print("Converged")
        else:
            print("Ideal log-likelihood reached, stopping to avoid overfitting")
        break


Initial log likelihood:  -39.036074155716136
Ideal log likelihood:  -31.17322041000491
Iteration 1 log-likelihood: -34.34883254506644
Iteration 2 log-likelihood: -33.789852385753115
Iteration 3 log-likelihood: -33.531102410495926
Iteration 4 log-likelihood: -33.362681325089525
Iteration 5 log-likelihood: -33.238355730546914
Iteration 6 log-likelihood: -33.14057934403694
Iteration 7 log-likelihood: -33.061087177593116
Iteration 8 log-likelihood: -32.995169990813444
Iteration 9 log-likelihood: -32.93964186727326
Iteration 10 log-likelihood: -32.89237368062056
Converged


## With initial conditions
The following code is assuming only a slight deviation in the gmm, hence estimated gmm is initialised as being close to the original gmm

In [49]:
# utility function to shift gmm
def shift_gmm(estimated_gmm, original_gmm, fc):
    # shift means randomly by fc% of its norm and copy everything else as is
    for i in range(n_components):
        estimated_gmm.means_[i] = original_gmm.means_[i] + (fc * np.sqrt(np.linalg.norm(original_gmm.means_[i])**2 / n_features) * np.random.randn(estimated_gmm.means_[i].shape[0]))
        estimated_gmm.covariances_[i] = original_gmm.covariances_[i]
        estimated_gmm.precisions_[i] = original_gmm.precisions_[i]
        estimated_gmm.precisions_cholesky_[i] = original_gmm.precisions_cholesky_[i]
        estimated_gmm.weights_ = original_gmm.weights_

    return estimated_gmm

In [50]:
# initialise estimate of the original gmm
# assumption is that we know the number of components required to model the original gmm
estimated_gmm = make_gmm(n_components, n_features, random_state=7)
estimated_gmm = shift_gmm(estimated_gmm,my_test_gmm, 0.1)
# for i in range(n_components):
#     # shift mean randomly by 10% of its norm
#     print(f"Initial mean {i}: {estimated_gmm.means_[i]}")
#     print(f"Original mean {i}: {my_test_gmm.means_[i]}")

# perform updates on the estimated gmm
# log likelihoods stores the log likelihoods of the estimated gmm computed on the samples drawn from the original gmm, our objective is to maximise this
tol = 0.05
log_likelihoods = []
y_gmms = form_y_gmms(estimated_gmm, measurement_matrices, noise_covariance_matrix)
log_likelihoods.append(get_log_likelihood(measurement_matrices, compressed_measurements, sample_measurement_matrix, y_gmms))

print("Initial log likelihood: ", log_likelihoods[-1])
ideal_log_likelihood = get_log_likelihood(measurement_matrices, compressed_measurements, sample_measurement_matrix, form_y_gmms(my_test_gmm, measurement_matrices, noise_covariance_matrix))
print("Ideal log likelihood: ", ideal_log_likelihood)

noise_inverse =  np.linalg.inv(noise_covariance_matrix)
iter_counter = 0

while True:

    new_weights = get_new_weights(compressed_measurements, y_gmms, sample_measurement_matrix)
    new_means = get_new_means(compressed_measurements, y_gmms, sample_measurement_matrix, estimated_gmm, measurement_matrices, noise_covariance_matrix)
    new_covariances = get_new_covariances(compressed_measurements, y_gmms, sample_measurement_matrix, estimated_gmm, measurement_matrices, noise_covariance_matrix, new_means)
    # estimated_gmm.weights_ = get_new_weights(compressed_measurements, y_gmms, sample_measurement_matrix)
    # estimated_gmm.means_ = get_new_means(compressed_measurements, y_gmms, sample_measurement_matrix, estimated_gmm, measurement_matrices, noise_covariance_matrix)
    # estimated_gmm.covariances_ = get_new_covariances(compressed_measurements, y_gmms, sample_measurement_matrix, estimated_gmm, measurement_matrices, noise_covariance_matrix, estimated_gmm.means_)
    estimated_gmm.weights_ = new_weights
    estimated_gmm.means_ = new_means
    estimated_gmm.covariances_ = new_covariances
    
    precision_array = [np.linalg.pinv(cov) for cov in estimated_gmm.covariances_]
    estimated_gmm.precisions_ = np.array(precision_array)
    estimated_gmm.precisions_cholesky_ = np.array([np.linalg.cholesky(prec) for prec in estimated_gmm.precisions_])

    y_gmms = form_y_gmms(estimated_gmm, measurement_matrices, noise_covariance_matrix)
    log_likelihoods.append(get_log_likelihood(measurement_matrices, compressed_measurements, sample_measurement_matrix, y_gmms))
    print(f"Iteration {iter_counter + 1} log-likelihood: {log_likelihoods[-1]}")
    iter_counter += 1
    if abs(log_likelihoods[-1] - log_likelihoods[-2]) < tol or log_likelihoods[-1] > ideal_log_likelihood:
        if abs(log_likelihoods[-1] - log_likelihoods[-2]) < tol:
            print("Converged")
            break
        # else:
        #     print("Ideal log-likelihood reached, stopping to avoid overfitting")


Initial log likelihood:  -46.28327186778595
Ideal log likelihood:  -41.666608115490384
Iteration 1 log-likelihood: -42.105632451875685
Iteration 2 log-likelihood: -41.82562429325525
Iteration 3 log-likelihood: -41.68822249015458
Iteration 4 log-likelihood: -41.60120457405397
Iteration 5 log-likelihood: -41.54266685957666
Iteration 6 log-likelihood: -41.50169443941766
Converged


In [54]:
# get test sample set, take compressive measurements, reconstruct and check rmse
n_test = 500
test_sample_set = my_test_gmm.sample(n_samples=n_test)[0]
test_compressed_measurements = []
test_sample_matrix_indices = []
for i in range(n_test):
    matrix_index = np.random.randint(0, len(measurement_matrices))
    test_sample_matrix_indices.append(matrix_index)
    test_compressed_measurements.append(measurement_matrices[matrix_index] @ test_sample_set[i])

test_reconstructed_samples = []
test_reconstructed_samples_with_original_gmm = []
y_gmms = form_y_gmms(estimated_gmm, measurement_matrices, noise_covariance_matrix)
y_gmms_orig = form_y_gmms(my_test_gmm, measurement_matrices, noise_covariance_matrix)

for i in range(n_test):
    y_gmm = y_gmms[test_sample_matrix_indices[i]]
    y_gmm_orig = y_gmms_orig[test_sample_matrix_indices[i]]
    reconstructed_sample = np.zeros(estimated_gmm.means_[0].shape)
    reconstructed_sample_orig = np.zeros(my_test_gmm.means_[0].shape)
    psum = 0
    psum_orig = 0
    for j in range(n_components):
        p_ik = y_gmm.weights_[j] * multivariate_normal.pdf(test_compressed_measurements[i], mean=y_gmm.means_[j], cov=y_gmm.covariances_[j]) / np.exp(y_gmm.score(test_compressed_measurements[i].reshape(1, -1)))
        p_ik_orig = y_gmm_orig.weights_[j] * multivariate_normal.pdf(test_compressed_measurements[i], mean=y_gmm_orig.means_[j], cov=y_gmm_orig.covariances_[j]) / np.exp(y_gmm_orig.score(test_compressed_measurements[i].reshape(1, -1)))
        psum += p_ik
        psum_orig += p_ik_orig
        n_ik = get_x_reconstruction(measurement_matrices[test_sample_matrix_indices[i]], noise_covariance_matrix, estimated_gmm.covariances_[j], estimated_gmm.means_[j], test_compressed_measurements[i])
        n_ik_orig = get_x_reconstruction(measurement_matrices[test_sample_matrix_indices[i]], noise_covariance_matrix, my_test_gmm.covariances_[j], my_test_gmm.means_[j], test_compressed_measurements[i])
        reconstructed_sample += p_ik * n_ik
        reconstructed_sample_orig += p_ik_orig * n_ik_orig
    assert abs(psum - 1) < 1e-3
    assert abs(psum_orig - 1) < 1e-3
    test_reconstructed_samples.append(reconstructed_sample)
    test_reconstructed_samples_with_original_gmm.append(reconstructed_sample_orig)

test_reconstructed_samples = np.array(test_reconstructed_samples)
test_reconstructed_samples_with_original_gmm = np.array(test_reconstructed_samples_with_original_gmm)
test_rmse = 0
for i in range(n_test):
    test_rmse += np.linalg.norm(test_reconstructed_samples[i] - test_sample_set[i]) / np.linalg.norm(test_sample_set[i])
test_rmse /= n_test
print(f"Test RMSE: {test_rmse}")

test_rmse_orig = 0
for i in range(n_test):
    test_rmse_orig += np.linalg.norm(test_reconstructed_samples_with_original_gmm[i] - test_sample_set[i]) / np.linalg.norm(test_sample_set[i])
test_rmse_orig /= n_test
print(f"Test RMSE with original GMM: {test_rmse_orig}")

Test RMSE: 0.058868173032122356
Test RMSE with original GMM: 0.05566008007988325


In [55]:
print(test_sample_set[0])
print(test_reconstructed_samples[0])

[  9.60439257  -4.98422776  -8.10066273   3.24903175  11.54513042
  -1.35877931  17.47838942   0.53470597   9.39891469  22.91456363
   1.06972214 -18.96815579   3.26234575  19.3436277   -4.31814903
 -11.14217858  -4.13033329  -2.07150076   2.08938565   6.97327998]
[  9.57991945  -5.24076535  -7.66024556   2.86350325  11.76178894
  -1.36240888  17.89306714   0.62983187   9.40577758  22.52490605
   0.9907406  -18.78789717   2.78753183  19.1747585   -4.4136155
 -10.68310346  -4.18305998  -1.95821821   2.39120026   7.22455477]
